# Notebook 02 -- Clustering & Pseudo-Label Generation

## Verizon Cell Tower Lease Fair Market Value Estimation

This notebook implements **Stage 1 (Comparable-Site Clustering)** and **Stage 2 (Pseudo-Label Generation)** of the FMV estimation pipeline.

### Pipeline Stages Covered

| Stage | Component | Description |
|-------|-----------|-------------|
| **1** | Clustering | Group sites into peer clusters using market features + HDBSCAN |
| **2** | Pseudo-Labeling | Assign CORE / MARGINAL / OUTLIER labels based on within-cluster rent deviation |

### Key Idea

We lack ground-truth FMV labels. The clustering + pseudo-labeling approach creates
training signal by identifying sites whose rents are **consistent with their market
peers** (CORE sites). These CORE sites become the supervised training set for the
FMV regression model in Notebook 03.

### Sections

| Section | Description |
|---------|-------------|
| 1. Setup & Data Load | Imports, load parquet, define feature sets, encode categoricals |
| 2. Stage 1: Clustering | StandardScaler, PCA exploration, HDBSCAN, K-Means comparison |
| 3. Cluster Profiles | Detailed analysis of top clusters |
| 4. Stage 2: Pseudo-Labeling | MAD-based labeling, validation checks |
| 5. Sensitivity Analysis | Vary MAD threshold, recommend best |
| 6. Save Outputs | Export clustered + labeled dataset |

---
## 1. Setup & Data Load

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import median_abs_deviation, pearsonr, spearmanr
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# HDBSCAN: try standalone package first, fall back to sklearn
try:
    from hdbscan import HDBSCAN
    HDBSCAN_SOURCE = 'hdbscan'
    print('Using standalone hdbscan package')
except ImportError:
    from sklearn.cluster import HDBSCAN
    HDBSCAN_SOURCE = 'sklearn'
    print('Using sklearn.cluster.HDBSCAN')

np.random.seed(42)

sns.set_style('whitegrid')
plt.rcParams.update({
    'figure.dpi': 120,
    'figure.facecolor': 'white',
    'axes.titlesize': 12,
    'axes.labelsize': 10,
    'xtick.labelsize': 9,
    'ytick.labelsize': 9,
    'legend.fontsize': 9,
})

PROP_TYPES = ['commercial', 'rooftop', 'vacant_land', 'residential',
              'government', 'agricultural', 'industrial']
PROP_COLORS = dict(zip(PROP_TYPES, sns.color_palette('Set2', len(PROP_TYPES))))

print('All imports loaded.')

In [ ]:
DATA_PATH = '../data/synthetic/synthetic_5k_sites.parquet'

df = pd.read_parquet(DATA_PATH)
print(f'Shape: {df.shape[0]:,} rows x {df.shape[1]} columns')
print(f'Property types: {df["property_type"].nunique()}')
print(f'States:         {df["state"].nunique()}')
print(f'Metro regions:  {df["metro_region"].nunique()}')
print(f'\nRent: median=${df["current_monthly_rent"].median():,.0f}, '
      f'range=[${df["current_monthly_rent"].min():,.0f}, '
      f'${df["current_monthly_rent"].max():,.0f}]')

### 1.1 Define Feature Sets

**X_MARKET** contains the 37 market/property/zoning/competition features used for
clustering. These capture the "comparability" dimensions -- two sites in the same
cluster should be genuine market peers.

Network features (Z_NETWORK) are excluded from clustering because they reflect
Verizon's internal deployment decisions rather than market conditions.

In [ ]:
# 37 market features used for clustering (same list as simulations)
X_MARKET = [
    # Market / Demographics (Domain 5)
    'census_population_density', 'census_population_3km', 'census_median_income',
    'census_median_home_value', 'census_median_rent', 'census_vacancy_rate',
    'urban_rural_class', 'underlying_land_value_psf', 'hud_safmr',
    'property_tax_rate', 'topography_class', 'terrain_elevation_variance_m',
    'poi_density_1km', 'building_density_500m', 'distance_to_highway_km',
    'ground_elevation_ft', 'unemployment_rate_local',
    # Competition / Supply (Domain 7)
    'fcc_tower_count_5km', 'fcc_tower_density_sqkm',
    'competitor_tower_distance_km', 'tower_per_capita',
    'scarcity_index', 'landlord_concentration', 'composite_hazard_score',
    # Property Characteristics (Domain 1 / Domain 3)
    'tower_height_ft', 'lease_area_sqft', 'parcel_acreage', 'construction_age',
    'site_access_ordinal', 'permit_approval_difficulty', 'wrluri_index',
    'colocation_required', 'height_restriction_ft',
    'has_building', 'is_ground_tower', 'has_parcel', 'has_zoning_data',
]

# Categoricals that need encoding
CAT_FEATURES = ['property_type', 'structure_type', 'zoning_district_type']

print(f'X_MARKET: {len(X_MARKET)} numeric features')
print(f'Categorical features to encode: {CAT_FEATURES}')

# Verify all features exist in dataframe
missing_feats = [f for f in X_MARKET if f not in df.columns]
if missing_feats:
    print(f'WARNING: Missing features: {missing_feats}')
else:
    print('All X_MARKET features found in dataframe.')

### 1.2 Encode Categoricals

- **property_type**: one-hot encoded (7 categories -> 7 binary columns)
- **structure_type**: label-encoded (ordinal has no natural order, but label encoding
  is sufficient for distance-based clustering after scaling)
- **zoning_district_type**: label-encoded

In [ ]:
# Make a working copy so we don't pollute the original dataframe
df_work = df.copy()

# Label-encode structure_type and zoning_district_type
le_structure = LabelEncoder()
le_zoning = LabelEncoder()
df_work['structure_type_enc'] = le_structure.fit_transform(df_work['structure_type'].astype(str))
df_work['zoning_district_type_enc'] = le_zoning.fit_transform(df_work['zoning_district_type'].astype(str))

print('Label Encodings:')
print(f'  structure_type:       {dict(zip(le_structure.classes_, le_structure.transform(le_structure.classes_)))}')
print(f'  zoning_district_type: {dict(zip(le_zoning.classes_, le_zoning.transform(le_zoning.classes_)))}')

# One-hot encode property_type
prop_dummies = pd.get_dummies(df_work['property_type'], prefix='pt')
df_work = pd.concat([df_work, prop_dummies], axis=1)

print(f'\nOne-hot columns created: {list(prop_dummies.columns)}')

# Build the full encoded feature list
ENCODED_CAT_COLS = ['structure_type_enc', 'zoning_district_type_enc'] + list(prop_dummies.columns)
X_MARKET_ENC = X_MARKET + ENCODED_CAT_COLS

print(f'\nX_MARKET_ENC: {len(X_MARKET_ENC)} total features '
      f'({len(X_MARKET)} numeric + {len(ENCODED_CAT_COLS)} encoded categorical)')

# Quick sanity check
assert all(c in df_work.columns for c in X_MARKET_ENC), 'Missing encoded columns!'
print('All encoded features verified in dataframe.')

---
## 2. Stage 1: Clustering

The goal is to group sites into **comparable-site clusters** -- peer groups where
sites share similar market conditions. Within each cluster, rent variation should
primarily reflect lease-level noise (escalation drift, origination differences)
rather than fundamental market differences.

**Key validation**: within-cluster rent CV should be significantly lower than
portfolio-wide CV.

### 2.1 StandardScaler & PCA Exploration

In [ ]:
# Scale all features to zero mean, unit variance
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_work[X_MARKET_ENC].values.astype(float))

print(f'Scaled feature matrix: {X_scaled.shape}')
print(f'Mean range:  [{X_scaled.mean(axis=0).min():.6f}, {X_scaled.mean(axis=0).max():.6f}]')
print(f'Std range:   [{X_scaled.std(axis=0).min():.4f}, {X_scaled.std(axis=0).max():.4f}]')

In [ ]:
# PCA: compute all components and examine explained variance
pca_full = PCA()
pca_full.fit(X_scaled)

cumvar = np.cumsum(pca_full.explained_variance_ratio_)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scree plot
axes[0].bar(range(1, len(pca_full.explained_variance_ratio_) + 1),
            pca_full.explained_variance_ratio_, color='steelblue', alpha=0.8)
axes[0].set_title('PCA Scree Plot')
axes[0].set_xlabel('Principal Component')
axes[0].set_ylabel('Explained Variance Ratio')
axes[0].set_xlim(0, min(50, len(pca_full.explained_variance_ratio_)))

# Cumulative variance
axes[1].plot(range(1, len(cumvar) + 1), cumvar, 'o-', markersize=3, color='coral')
axes[1].axhline(0.90, color='grey', ls='--', lw=1, label='90% variance')
axes[1].axhline(0.95, color='grey', ls=':', lw=1, label='95% variance')

# Mark the number of components for 90% and 95%
n_90 = np.searchsorted(cumvar, 0.90) + 1
n_95 = np.searchsorted(cumvar, 0.95) + 1
axes[1].axvline(n_90, color='red', ls='--', lw=1, alpha=0.7)
axes[1].axvline(n_95, color='red', ls=':', lw=1, alpha=0.7)
axes[1].annotate(f'{n_90} PCs = 90%', xy=(n_90, 0.90), fontsize=9,
                 xytext=(n_90 + 2, 0.85),
                 arrowprops=dict(arrowstyle='->', color='red'))
axes[1].annotate(f'{n_95} PCs = 95%', xy=(n_95, 0.95), fontsize=9,
                 xytext=(n_95 + 2, 0.92),
                 arrowprops=dict(arrowstyle='->', color='red'))

axes[1].set_title('Cumulative Explained Variance')
axes[1].set_xlabel('Number of Principal Components')
axes[1].set_ylabel('Cumulative Variance Explained')
axes[1].legend(fontsize=8)

plt.suptitle('PCA Exploration: Dimensionality of Market Feature Space', fontsize=13, y=1.02)
plt.tight_layout()
plt.show()

print(f'Components for 90% variance: {n_90}/{X_scaled.shape[1]}')
print(f'Components for 95% variance: {n_95}/{X_scaled.shape[1]}')
print(f'\nDecision: {"PCA reduction not needed -- HDBSCAN handles high-D well enough" if n_90 > X_scaled.shape[1] * 0.5 else f"Consider PCA to {n_90} components for noise reduction"}')

### 2.2 HDBSCAN Clustering

HDBSCAN is our primary clustering algorithm because:
- It discovers the number of clusters automatically
- It handles varying cluster densities (urban vs rural markets)
- It identifies noise points that don't fit any cluster
- It produces a hierarchy of clusters that can be tuned

Parameters from config: `min_cluster_size=20`, `min_samples=5`

In [ ]:
# Run HDBSCAN
hdbscan_model = HDBSCAN(min_cluster_size=20, min_samples=5)
hdbscan_labels = hdbscan_model.fit_predict(X_scaled)

df_work['hdbscan_cluster'] = hdbscan_labels

n_clusters = len(set(hdbscan_labels)) - (1 if -1 in hdbscan_labels else 0)
n_noise = (hdbscan_labels == -1).sum()
noise_pct = n_noise / len(df_work) * 100

print('=' * 60)
print('HDBSCAN Clustering Results')
print('=' * 60)
print(f'Number of clusters:     {n_clusters}')
print(f'Noise points:           {n_noise} ({noise_pct:.1f}%)')
print(f'Clustered points:       {len(df_work) - n_noise} ({100 - noise_pct:.1f}%)')

# Cluster size distribution
cluster_sizes = pd.Series(hdbscan_labels[hdbscan_labels >= 0]).value_counts().sort_values(ascending=False)
print(f'\nCluster size distribution:')
print(f'  Min:    {cluster_sizes.min()}')
print(f'  Median: {cluster_sizes.median():.0f}')
print(f'  Mean:   {cluster_sizes.mean():.1f}')
print(f'  Max:    {cluster_sizes.max()}')
print(f'  Std:    {cluster_sizes.std():.1f}')

In [ ]:
# Silhouette score (excluding noise points)
non_noise_mask = hdbscan_labels >= 0
if non_noise_mask.sum() > 1 and n_clusters >= 2:
    sil_hdbscan = silhouette_score(X_scaled[non_noise_mask], hdbscan_labels[non_noise_mask])
    print(f'HDBSCAN Silhouette Score (excl. noise): {sil_hdbscan:.4f}')
else:
    sil_hdbscan = np.nan
    print('Cannot compute silhouette (too few clusters or points)')

### 2.3 KEY Validation: Within-Cluster Rent CV vs Portfolio-Wide CV

This is the **most important validation** of clustering quality. If clusters
are meaningful peer groups, the coefficient of variation (CV) of rent **within**
clusters should be substantially lower than the portfolio-wide CV. A lower
within-cluster CV means clusters successfully group sites with similar FMV.

In [ ]:
portfolio_cv = df_work['current_monthly_rent'].std() / df_work['current_monthly_rent'].mean()

# Within-cluster CV for each cluster
cluster_cv_list = []
for cid in sorted(df_work.loc[non_noise_mask, 'hdbscan_cluster'].unique()):
    rents = df_work.loc[df_work['hdbscan_cluster'] == cid, 'current_monthly_rent']
    if len(rents) >= 5 and rents.mean() > 0:
        cv = rents.std() / rents.mean()
        cluster_cv_list.append({'cluster_id': cid, 'n': len(rents), 'cv': cv,
                                 'median_rent': rents.median()})

cv_df = pd.DataFrame(cluster_cv_list)

# Weighted average within-cluster CV (weighted by cluster size)
weighted_cv = np.average(cv_df['cv'], weights=cv_df['n'])

print('=' * 60)
print('CLUSTERING QUALITY: Rent CV Comparison')
print('=' * 60)
print(f'Portfolio-wide CV:             {portfolio_cv:.4f}')
print(f'Weighted avg within-cluster CV: {weighted_cv:.4f}')
print(f'CV reduction:                  {(1 - weighted_cv / portfolio_cv) * 100:.1f}%')
print()

if weighted_cv < portfolio_cv * 0.8:
    print('PASS: Clustering achieves meaningful rent homogeneity within clusters.')
    print(f'  Within-cluster CV is {(1 - weighted_cv / portfolio_cv) * 100:.1f}% lower than portfolio CV.')
else:
    print('WARNING: Within-cluster CV is not substantially lower than portfolio CV.')
    print('  Clusters may not be capturing market similarity effectively.')

print(f'\nCluster CV distribution:')
print(f'  Min CV:    {cv_df["cv"].min():.4f}')
print(f'  Median CV: {cv_df["cv"].median():.4f}')
print(f'  Max CV:    {cv_df["cv"].max():.4f}')

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 1. Cluster size histogram
axes[0].hist(cluster_sizes.values, bins=30, color='steelblue', edgecolor='white', alpha=0.8)
axes[0].axvline(cluster_sizes.median(), color='red', ls='--', lw=1.5,
                label=f'Median = {cluster_sizes.median():.0f}')
axes[0].set_title(f'Cluster Size Distribution (n={n_clusters} clusters)')
axes[0].set_xlabel('Cluster Size')
axes[0].set_ylabel('Frequency')
axes[0].legend()

# 2. Boxplots of rent by cluster (top 10 largest clusters)
top_10_clusters = cluster_sizes.head(10).index.tolist()
df_top10 = df_work[df_work['hdbscan_cluster'].isin(top_10_clusters)].copy()
df_top10['hdbscan_cluster'] = df_top10['hdbscan_cluster'].astype(str)
order_top10 = [str(c) for c in top_10_clusters]
sns.boxplot(data=df_top10, x='hdbscan_cluster', y='current_monthly_rent',
            order=order_top10, palette='Set2', ax=axes[1],
            flierprops=dict(marker='o', markersize=2, alpha=0.3))
axes[1].set_title('Rent Distribution by Cluster (Top 10 Largest)')
axes[1].set_xlabel('Cluster ID')
axes[1].set_ylabel('Current Monthly Rent ($/mo)')
axes[1].tick_params(axis='x', rotation=45)

# Add cluster sizes as labels
for i, cid in enumerate(top_10_clusters):
    n = cluster_sizes[cid]
    axes[1].text(i, axes[1].get_ylim()[1] * 0.98, f'n={n}',
                 ha='center', va='top', fontsize=7, color='red')

# 3. Scatter of sites colored by cluster
# Use a limited palette for readability; noise in grey
noise_mask = df_work['hdbscan_cluster'] == -1
clustered = df_work[~noise_mask]

axes[2].scatter(df_work.loc[noise_mask, 'lon'], df_work.loc[noise_mask, 'lat'],
                s=3, alpha=0.15, c='grey', label=f'Noise (n={noise_mask.sum()})')
sc = axes[2].scatter(clustered['lon'], clustered['lat'],
                     c=clustered['hdbscan_cluster'], cmap='tab20',
                     s=5, alpha=0.5)
axes[2].set_title('Sites Colored by HDBSCAN Cluster')
axes[2].set_xlabel('Longitude')
axes[2].set_ylabel('Latitude')
axes[2].set_xlim(-130, -65)
axes[2].set_ylim(24, 50)
axes[2].set_aspect('equal')
axes[2].legend(fontsize=7, loc='lower left')

plt.suptitle('Stage 1: HDBSCAN Clustering Results', fontsize=13, y=1.02)
plt.tight_layout()
plt.show()

### 2.4 Comparison: K-Means with k=25

We run K-Means as a baseline comparison to validate that HDBSCAN provides
meaningful improvements in clustering quality.

In [ ]:
# K-Means with k=25
kmeans = KMeans(n_clusters=25, random_state=42, n_init=10, max_iter=300)
kmeans_labels = kmeans.fit_predict(X_scaled)

df_work['kmeans_cluster'] = kmeans_labels

# K-Means silhouette
sil_kmeans = silhouette_score(X_scaled, kmeans_labels)

# K-Means within-cluster CV
km_cv_list = []
for cid in range(25):
    rents = df_work.loc[df_work['kmeans_cluster'] == cid, 'current_monthly_rent']
    if len(rents) >= 5 and rents.mean() > 0:
        km_cv_list.append({'cluster_id': cid, 'n': len(rents),
                            'cv': rents.std() / rents.mean()})

km_cv_df = pd.DataFrame(km_cv_list)
km_weighted_cv = np.average(km_cv_df['cv'], weights=km_cv_df['n'])

print('=' * 60)
print('CLUSTERING COMPARISON: HDBSCAN vs K-Means')
print('=' * 60)
print(f'{"Metric":<35s} {"HDBSCAN":>12s} {"K-Means":>12s}')
print('-' * 60)
print(f'{"Clusters":35s} {n_clusters:12d} {25:12d}')
print(f'{"Noise points":35s} {n_noise:12d} {0:12d}')
print(f'{"Silhouette score":35s} {sil_hdbscan:12.4f} {sil_kmeans:12.4f}')
print(f'{"Weighted within-cluster CV":35s} {weighted_cv:12.4f} {km_weighted_cv:12.4f}')
print(f'{"Portfolio CV":35s} {portfolio_cv:12.4f} {portfolio_cv:12.4f}')
print(f'{"CV reduction vs portfolio":35s} {(1-weighted_cv/portfolio_cv)*100:11.1f}% {(1-km_weighted_cv/portfolio_cv)*100:11.1f}%')
print()

if weighted_cv < km_weighted_cv:
    print('=> HDBSCAN achieves better within-cluster rent homogeneity.')
else:
    print('=> K-Means achieves comparable or better rent homogeneity.')
    print('   However, HDBSCAN identifies noise points, which is valuable for outlier detection.')

---
## 3. Cluster Profile Analysis

For the top 5 largest clusters, we examine:
- Feature centroids (mean of key market features)
- Property type composition
- Rent distribution

This validates that clusters represent **meaningful market segments**.

In [ ]:
top_5_clusters = cluster_sizes.head(5).index.tolist()

profile_features = [
    'census_population_density', 'census_median_home_value',
    'underlying_land_value_psf', 'census_median_income',
    'fcc_tower_count_5km', 'urban_rural_class',
    'current_monthly_rent',
]

for cid in top_5_clusters:
    cluster_data = df_work[df_work['hdbscan_cluster'] == cid]
    n = len(cluster_data)
    
    print('=' * 70)
    print(f'CLUSTER {cid}  (n={n} sites)')
    print('=' * 70)
    
    # Feature centroids
    print('\nFeature Centroids:')
    for feat in profile_features:
        mean_val = cluster_data[feat].mean()
        median_val = cluster_data[feat].median()
        portfolio_median = df_work[feat].median()
        ratio = median_val / portfolio_median if portfolio_median != 0 else np.nan
        print(f'  {feat:35s}: median={median_val:10.1f}  '
              f'(vs portfolio: {ratio:.2f}x)')
    
    # Property type mix
    pt_mix = cluster_data['property_type'].value_counts()
    print(f'\nProperty Type Mix:')
    for pt, count in pt_mix.items():
        print(f'  {pt:20s}: {count:4d} ({count/n*100:5.1f}%)')
    
    # Rent distribution
    rent = cluster_data['current_monthly_rent']
    cv = rent.std() / rent.mean() if rent.mean() > 0 else np.nan
    print(f'\nRent: median=${rent.median():,.0f}, mean=${rent.mean():,.0f}, '
          f'CV={cv:.3f}, range=[${rent.min():,.0f}, ${rent.max():,.0f}]')
    
    # Geographic spread
    states = cluster_data['state'].nunique()
    metros = cluster_data['metro_region'].nunique()
    print(f'Geography: {states} states, {metros} metro regions')
    print()

In [ ]:
# Visual: rent distributions for top 5 clusters
fig, axes = plt.subplots(1, 5, figsize=(20, 4), sharey=True)

colors = sns.color_palette('Set2', 5)
for i, cid in enumerate(top_5_clusters):
    rents = df_work.loc[df_work['hdbscan_cluster'] == cid, 'current_monthly_rent']
    axes[i].hist(rents, bins=30, color=colors[i], edgecolor='white', alpha=0.8)
    axes[i].axvline(rents.median(), color='red', ls='--', lw=1.5)
    cv = rents.std() / rents.mean() if rents.mean() > 0 else np.nan
    axes[i].set_title(f'Cluster {cid}\nn={len(rents)}, CV={cv:.3f}', fontsize=10)
    axes[i].set_xlabel('$/mo')
    if i == 0:
        axes[i].set_ylabel('Frequency')

plt.suptitle('Rent Distributions: Top 5 Largest Clusters', fontsize=13, y=1.05)
plt.tight_layout()
plt.show()

---
## 4. Stage 2: Pseudo-Label Generation

For each cluster, we compute the **Median Absolute Deviation (MAD)** of rent and
assign labels based on how far each site's rent deviates from the cluster median:

| Label | Condition | Interpretation |
|-------|-----------|----------------|
| **CORE** | z < 1.0 | Rent is consistent with market peers -- likely near FMV |
| **MARGINAL** | 1.0 <= z < 2.0 | Moderately deviant -- could be slightly over/under |
| **OUTLIER** | z >= 2.0 | Strongly deviant -- likely distorted by escalation drift |

where $z = |\text{rent} - \text{median}| / \text{MAD}$

Noise points (cluster = -1) are labeled as MARGINAL by default.

In [ ]:
MAD_CORE_THRESHOLD = 1.0
MAD_OUTLIER_THRESHOLD = 2.0

def assign_pseudo_labels(df_in, cluster_col, core_thresh=1.0, outlier_thresh=2.0):
    """
    Assign CORE / MARGINAL / OUTLIER pseudo-labels based on within-cluster
    MAD-normalised deviation of current_monthly_rent.
    
    Parameters
    ----------
    df_in : DataFrame with cluster_col and 'current_monthly_rent'
    cluster_col : name of cluster column
    core_thresh : z < this -> CORE
    outlier_thresh : z >= this -> OUTLIER
    
    Returns
    -------
    Series of labels, Series of z-scores
    """
    labels = pd.Series('MARGINAL', index=df_in.index)
    z_scores = pd.Series(np.nan, index=df_in.index)
    
    for cid in df_in[cluster_col].unique():
        if cid == -1:
            continue  # noise stays MARGINAL
        mask = df_in[cluster_col] == cid
        rents = df_in.loc[mask, 'current_monthly_rent']
        med = rents.median()
        mad = median_abs_deviation(rents, scale='normal')
        if mad == 0:
            mad = rents.std()
        if mad == 0 or np.isnan(mad):
            mad = 1.0  # fallback for degenerate clusters
        
        z = np.abs(rents - med) / mad
        z_scores.loc[mask] = z
        labels.loc[mask & (z < core_thresh)] = 'CORE'
        labels.loc[mask & (z >= outlier_thresh)] = 'OUTLIER'
    
    return labels, z_scores

# Apply pseudo-labeling
df_work['pseudo_label'], df_work['mad_z_score'] = assign_pseudo_labels(
    df_work, 'hdbscan_cluster', MAD_CORE_THRESHOLD, MAD_OUTLIER_THRESHOLD
)

# Summary
label_counts = df_work['pseudo_label'].value_counts()
label_pcts = df_work['pseudo_label'].value_counts(normalize=True) * 100

print('=' * 60)
print('PSEUDO-LABEL DISTRIBUTION')
print('=' * 60)
for lbl in ['CORE', 'MARGINAL', 'OUTLIER']:
    n = label_counts.get(lbl, 0)
    pct = label_pcts.get(lbl, 0)
    print(f'  {lbl:10s}: {n:5d} sites ({pct:5.1f}%)')

core_pct = label_pcts.get('CORE', 0)
print(f'\nCore coverage: {core_pct:.1f}%')
if 40 <= core_pct <= 60:
    print('PASS: Core coverage is within target range [40%, 60%].')
elif core_pct < 40:
    print('WARNING: Core coverage below 40% -- consider lowering MAD threshold.')
else:
    print('WARNING: Core coverage above 60% -- consider raising MAD threshold.')

In [ ]:
# Label distribution by property type
label_by_pt = pd.crosstab(df_work['property_type'], df_work['pseudo_label'],
                            normalize='index') * 100
label_by_pt = label_by_pt[['CORE', 'MARGINAL', 'OUTLIER']]

print('Pseudo-Label Distribution by Property Type (%):')
print(label_by_pt.round(1).to_string())
print()

# Check: is any property type severely underrepresented in CORE?
underrep = label_by_pt[label_by_pt['CORE'] < 30]
if len(underrep) > 0:
    print(f'WARNING: {len(underrep)} property type(s) have <30% CORE representation:')
    for pt in underrep.index:
        print(f'  {pt}: {underrep.loc[pt, "CORE"]:.1f}% CORE')
else:
    print('All property types have adequate CORE representation (>= 30%).')

### 4.1 Validation: Geographic Diversity of CORE Sites

CORE sites should be geographically diverse -- not concentrated in one region.
If all CORE sites are from a few states, the supervised model will learn
geographically biased patterns.

In [ ]:
core_mask = df_work['pseudo_label'] == 'CORE'
core_states = df_work.loc[core_mask, 'state'].nunique()
total_states = df_work['state'].nunique()

print('=' * 60)
print('GEOGRAPHIC DIVERSITY CHECK')
print('=' * 60)
print(f'States with CORE sites: {core_states} / {total_states} total')
print(f'Coverage: {core_states / total_states * 100:.1f}%')

# State-level CORE percentage
state_core = (df_work.groupby('state')
              .apply(lambda x: (x['pseudo_label'] == 'CORE').mean() * 100)
              .sort_values(ascending=True))

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Bar chart: CORE % by state
colors = plt.cm.RdYlGn(state_core.values / 100)
axes[0].barh(range(len(state_core)), state_core.values, color=colors)
axes[0].set_yticks(range(len(state_core)))
axes[0].set_yticklabels(state_core.index, fontsize=8)
axes[0].set_xlabel('CORE %')
axes[0].set_title('CORE Site Percentage by State')
axes[0].axvline(40, color='red', ls='--', lw=1, alpha=0.7, label='40% target')
axes[0].legend(fontsize=8)

# Map: CORE vs non-CORE
label_colors = {'CORE': '#2ecc71', 'MARGINAL': '#f39c12', 'OUTLIER': '#e74c3c'}
for lbl in ['OUTLIER', 'MARGINAL', 'CORE']:  # draw CORE last for visibility
    mask = df_work['pseudo_label'] == lbl
    axes[1].scatter(df_work.loc[mask, 'lon'], df_work.loc[mask, 'lat'],
                    s=3 if lbl != 'CORE' else 5,
                    alpha=0.3 if lbl != 'CORE' else 0.5,
                    c=label_colors[lbl], label=f'{lbl} (n={mask.sum()})')
axes[1].set_title('Geographic Distribution of Pseudo-Labels')
axes[1].set_xlabel('Longitude')
axes[1].set_ylabel('Latitude')
axes[1].set_xlim(-130, -65)
axes[1].set_ylim(24, 50)
axes[1].set_aspect('equal')
axes[1].legend(fontsize=7, loc='lower left')

plt.suptitle('Stage 2: Pseudo-Label Geographic Diversity', fontsize=13, y=1.02)
plt.tight_layout()
plt.show()

### 4.2 Validation: CORE Site Rents vs Land Value Correlation

If CORE sites are truly near-FMV, their rents should correlate well with the
underlying land value (since the FMV generating function includes land value
as a key predictor). We target Pearson r > 0.5 in log-space.

In [ ]:
core_data = df_work[core_mask].copy()

log_rent_core = np.log(core_data['current_monthly_rent'])
log_land_core = np.log1p(core_data['underlying_land_value_psf'])

r_pearson, p_pearson = pearsonr(log_land_core, log_rent_core)
r_spearman, p_spearman = spearmanr(core_data['underlying_land_value_psf'],
                                     core_data['current_monthly_rent'])

print('=' * 60)
print('CORE SITE CORRELATION CHECK')
print('=' * 60)
print(f'Pearson r (log-log):  {r_pearson:.4f}  (p={p_pearson:.2e})')
print(f'Spearman rho:         {r_spearman:.4f}  (p={p_spearman:.2e})')

if r_pearson > 0.5:
    print(f'\nPASS: CORE rents correlate well with land value (r={r_pearson:.3f} > 0.5).')
    print('  This supports the hypothesis that CORE sites are near-FMV.')
elif r_pearson > 0.3:
    print(f'\nMODERATE: Correlation is positive but below target (r={r_pearson:.3f}).')
    print('  CORE labeling is directionally correct but may include noisy sites.')
else:
    print(f'\nWARNING: Low correlation (r={r_pearson:.3f}). CORE labeling may be poor.')

# Scatter plot
fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(log_land_core, log_rent_core, s=5, alpha=0.3, color='#2ecc71')

# Regression line
m, b = np.polyfit(log_land_core, log_rent_core, 1)
x_line = np.linspace(log_land_core.min(), log_land_core.max(), 100)
ax.plot(x_line, m * x_line + b, 'r-', lw=2, label=f'OLS: slope={m:.3f}')

ax.annotate(f'Pearson r = {r_pearson:.3f}\nSpearman rho = {r_spearman:.3f}\nn = {len(core_data):,}',
            xy=(0.05, 0.92), xycoords='axes fraction', fontsize=10,
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

ax.set_title('CORE Sites: log(Rent) vs log(Land Value)')
ax.set_xlabel('log(1 + Land Value $/sqft)')
ax.set_ylabel('log(Current Monthly Rent)')
ax.legend()
plt.tight_layout()
plt.show()

---
## 5. Sensitivity Analysis

The MAD threshold controls the trade-off between **label purity** (how close CORE
sites are to cluster median) and **coverage** (how many CORE sites we get). We
sweep across three thresholds and compare.

| Threshold | Expected Effect |
|-----------|----------------|
| 0.7 | Stricter -- fewer CORE sites, higher purity |
| 1.0 | Baseline (current) |
| 1.3 | Looser -- more CORE sites, lower purity |

In [ ]:
thresholds = [0.7, 1.0, 1.3]
sensitivity_results = []

for thresh in thresholds:
    labels, _ = assign_pseudo_labels(
        df_work, 'hdbscan_cluster',
        core_thresh=thresh, outlier_thresh=thresh + 1.0
    )
    
    core_mask_t = labels == 'CORE'
    core_pct_t = core_mask_t.mean() * 100
    
    # Property type coverage: all 7 types should have >= 20% CORE
    pt_core_pcts = []
    for pt in PROP_TYPES:
        pt_mask = df_work['property_type'] == pt
        pt_core = (labels[pt_mask] == 'CORE').mean() * 100
        pt_core_pcts.append(pt_core)
    min_pt_core = min(pt_core_pcts)
    types_above_20 = sum(1 for p in pt_core_pcts if p >= 20)
    
    # Geographic coverage
    core_states_t = df_work.loc[core_mask_t, 'state'].nunique()
    total_states_t = df_work['state'].nunique()
    
    # Rent-land correlation for CORE sites
    if core_mask_t.sum() > 10:
        r_t, _ = pearsonr(
            np.log1p(df_work.loc[core_mask_t, 'underlying_land_value_psf']),
            np.log(df_work.loc[core_mask_t, 'current_monthly_rent'])
        )
    else:
        r_t = np.nan
    
    sensitivity_results.append({
        'MAD Threshold': thresh,
        'CORE %': round(core_pct_t, 1),
        'CORE n': core_mask_t.sum(),
        'Min PT CORE %': round(min_pt_core, 1),
        'PT Types >= 20%': types_above_20,
        'States w/ CORE': core_states_t,
        'State Coverage %': round(core_states_t / total_states_t * 100, 1),
        'Pearson r (log)': round(r_t, 4),
    })

sens_df = pd.DataFrame(sensitivity_results).set_index('MAD Threshold')

print('=' * 70)
print('SENSITIVITY ANALYSIS: MAD Threshold Sweep')
print('=' * 70)
print(sens_df.to_string())

In [ ]:
# Visual comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 1. CORE % vs threshold
axes[0].bar([str(t) for t in thresholds],
            [r['CORE %'] for r in sensitivity_results],
            color=['#e74c3c', '#2ecc71', '#3498db'], alpha=0.8)
axes[0].axhline(40, color='grey', ls='--', lw=1, label='40% target')
axes[0].axhline(60, color='grey', ls=':', lw=1, label='60% upper')
axes[0].set_title('CORE Coverage vs MAD Threshold')
axes[0].set_xlabel('MAD Threshold')
axes[0].set_ylabel('CORE %')
axes[0].legend(fontsize=8)

# 2. Pearson r vs threshold
axes[1].bar([str(t) for t in thresholds],
            [r['Pearson r (log)'] for r in sensitivity_results],
            color=['#e74c3c', '#2ecc71', '#3498db'], alpha=0.8)
axes[1].axhline(0.5, color='grey', ls='--', lw=1, label='r=0.5 target')
axes[1].set_title('CORE Rent-Land Correlation vs MAD Threshold')
axes[1].set_xlabel('MAD Threshold')
axes[1].set_ylabel('Pearson r')
axes[1].legend(fontsize=8)

# 3. Property type coverage vs threshold
axes[2].bar([str(t) for t in thresholds],
            [r['PT Types >= 20%'] for r in sensitivity_results],
            color=['#e74c3c', '#2ecc71', '#3498db'], alpha=0.8)
axes[2].axhline(7, color='grey', ls='--', lw=1, label='All 7 types')
axes[2].set_title('Property Types with >= 20% CORE')
axes[2].set_xlabel('MAD Threshold')
axes[2].set_ylabel('Number of Types')
axes[2].set_ylim(0, 8)
axes[2].legend(fontsize=8)

plt.suptitle('Sensitivity Analysis: MAD Threshold Impact', fontsize=13, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Recommend best threshold
print('=' * 60)
print('THRESHOLD RECOMMENDATION')
print('=' * 60)

best_thresh = None
best_score = -1

for r in sensitivity_results:
    # Scoring: core% in [40, 60] range, high correlation, full type coverage
    core_in_range = 1.0 if 40 <= r['CORE %'] <= 60 else 0.5
    corr_score = min(r['Pearson r (log)'] / 0.5, 1.0) if not np.isnan(r['Pearson r (log)']) else 0
    type_score = r['PT Types >= 20%'] / 7
    geo_score = r['State Coverage %'] / 100
    
    composite = core_in_range * 0.35 + corr_score * 0.30 + type_score * 0.20 + geo_score * 0.15
    
    print(f'  Threshold {r["MAD Threshold"]}: score = {composite:.3f} '
          f'(core_range={core_in_range:.1f}, corr={corr_score:.2f}, '
          f'type={type_score:.2f}, geo={geo_score:.2f})')
    
    if composite > best_score:
        best_score = composite
        best_thresh = r['MAD Threshold']

print(f'\nRECOMMENDED THRESHOLD: {best_thresh}')
print(f'  Composite score: {best_score:.3f}')

In [ ]:
# Re-apply with best threshold if different from default
if best_thresh != MAD_CORE_THRESHOLD:
    print(f'Re-applying pseudo-labels with threshold = {best_thresh}...')
    df_work['pseudo_label'], df_work['mad_z_score'] = assign_pseudo_labels(
        df_work, 'hdbscan_cluster',
        core_thresh=best_thresh, outlier_thresh=best_thresh + 1.0
    )
    label_counts = df_work['pseudo_label'].value_counts()
    for lbl in ['CORE', 'MARGINAL', 'OUTLIER']:
        n = label_counts.get(lbl, 0)
        pct = n / len(df_work) * 100
        print(f'  {lbl:10s}: {n:5d} ({pct:5.1f}%)')
else:
    print(f'Keeping default threshold = {MAD_CORE_THRESHOLD} (already optimal).')

---
## 6. Save Outputs

We save the complete dataframe with clustering and pseudo-label columns appended.
This file becomes the input for Notebook 03 (FMV Modeling).

In [ ]:
# Select columns to save
# Keep original columns + clustering/labeling columns + encoded features
output_cols = list(df.columns) + [
    'hdbscan_cluster', 'kmeans_cluster',
    'pseudo_label', 'mad_z_score',
    'structure_type_enc', 'zoning_district_type_enc',
] + list(prop_dummies.columns)

# Remove any duplicates
output_cols = list(dict.fromkeys(output_cols))

df_output = df_work[output_cols].copy()

out_dir = os.path.join('..', 'data', 'processed')
os.makedirs(out_dir, exist_ok=True)
out_path = os.path.join(out_dir, 'clustered_pseudolabeled.parquet')

df_output.to_parquet(out_path, index=False)
file_size_kb = os.path.getsize(out_path) / 1024

print('=' * 60)
print('OUTPUT SAVED')
print('=' * 60)
print(f'Path:    {out_path}')
print(f'Shape:   {df_output.shape[0]:,} rows x {df_output.shape[1]} columns')
print(f'Size:    {file_size_kb:.0f} KB')
print(f'\nNew columns added:')
new_cols = [c for c in df_output.columns if c not in df.columns]
for col in new_cols:
    print(f'  - {col}')

In [ ]:
# Final summary
core_final = (df_output['pseudo_label'] == 'CORE').sum()
marginal_final = (df_output['pseudo_label'] == 'MARGINAL').sum()
outlier_final = (df_output['pseudo_label'] == 'OUTLIER').sum()

print('=' * 70)
print('NOTEBOOK 02 -- FINAL SUMMARY')
print('=' * 70)
print(f'''
STAGE 1: CLUSTERING
  Algorithm:          HDBSCAN (min_cluster_size=20, min_samples=5)
  Clusters found:     {n_clusters}
  Noise points:       {n_noise} ({noise_pct:.1f}%)
  Silhouette score:   {sil_hdbscan:.4f}
  Portfolio-wide CV:  {portfolio_cv:.4f}
  Within-cluster CV:  {weighted_cv:.4f} (reduction: {(1-weighted_cv/portfolio_cv)*100:.1f}%)

STAGE 2: PSEUDO-LABELING
  MAD threshold:      {best_thresh}
  CORE sites:         {core_final:,} ({core_final/len(df_output)*100:.1f}%)
  MARGINAL sites:     {marginal_final:,} ({marginal_final/len(df_output)*100:.1f}%)
  OUTLIER sites:      {outlier_final:,} ({outlier_final/len(df_output)*100:.1f}%)
  Geographic coverage: {df_output.loc[df_output["pseudo_label"]=="CORE", "state"].nunique()}/{df_output["state"].nunique()} states
  Land-value corr:    r = {r_pearson:.3f}

OUTPUT
  Saved to: {out_path}
  Shape:    {df_output.shape}

=> PROCEED TO NOTEBOOK 03 (FMV Modeling)
''')